**OPTION:**  
 A contract that gives the buyer the right, but not the obligation, to buy or sell an underlying asset or instrument at a specific strike price prior to or on a specific date.

  **Premium:**  
    The price that the buyer pays for the option.

  **Expiration date:**  
    The fixed day when the option can be exercised.

  **Underlying:**  
    The asset on which the option is based.


**BINOMIAL TREE:** 

Time step duratio is calculated dividing the total time with the number of steps.
$$
dt = \frac{T}{N}
$$

Upside and Downsie factor:
$$
\text{e: euler base is due to assets move in exponential way.}
$$
$$
\sigma \text{: is the volatility of the asset.}
$$

$$
u = e^{\sigma \sqrt{dt}}
$$
$$
d = e^{-\sigma \sqrt{dt}}
$$

$$
\text{neutral risk upside probability:  } \frac{e^{r*dt}-d}{u-d}
$$
The tree can be represented as:  
$$
 S_{i,j} = S_0 \cdot u^{i-j} \cdot d^j
$$


S_0 = inital stock price

i = total steps

j = number of times that stock have fallen


In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [4]:
S = 100        
K = 100        
T = 1          
r = 0.05       
sigma = 0.2    
N = 3        
dt = T / N

u = np.exp(sigma * np.sqrt(dt))
d = np.exp(-sigma * np.sqrt(dt))

p = (np.exp(r * dt) - d) / (u - d)


print(f"dt = {dt:.4f}")
print(f"u = {u:.4f}")
print(f"d = {d:.4f}")
print(f"p = {p:.4f}")

dt = 0.3333
u = 1.1224
d = 0.8909
p = 0.5438


In [5]:
stock_tree = np.zeros((N+1, N+1))

for i in range(N+1):
    for j in range(i+1):
        stock_tree[j, i] = S*(u**(i-j))*((d**j))

stock_tree = pd.DataFrame(stock_tree)

print(stock_tree)

       0           1           2           3
0  100.0  112.240090  125.978379  141.398246
1    0.0   89.094725  100.000000  112.240090
2    0.0    0.000000   79.378701   89.094725
3    0.0    0.000000    0.000000   70.722235


Next, we are going to calculate the payoff for each option:
$$
\text{Payoff} = max(S_{T}-K,0)
$$

In [6]:
option_tree = np.zeros((N+1,N+1))

for i in range(N+1):
    option_tree[i, N] = max(stock_tree.iloc[i,N]-K,0)

option_tree = pd.DataFrame(option_tree)
print(option_tree)

print(option_tree.iloc[:, N])

     0    1    2          3
0  0.0  0.0  0.0  41.398246
1  0.0  0.0  0.0  12.240090
2  0.0  0.0  0.0   0.000000
3  0.0  0.0  0.0   0.000000
0    41.398246
1    12.240090
2     0.000000
3     0.000000
Name: 3, dtype: float64


**Backward Induction:** Is the discounted expected value of its two possible paths, where $C_{i,j}$ is the value in each node

$$
C_{i,j} = e^{-r \cdot dt}\cdot (p \cdot C_{i,j+1}+(1-p)\cdot C_{i+1,j+1})
$$


In [7]:
option_tree = option_tree.to_numpy()

for j in reversed(range(N)):
    for i in range(j+1):
        option_tree[i, j] = np.exp(-r * dt) * (p * option_tree[i, j+1] + (1 - p) * option_tree[i+1, j+1])


option_tree = pd.DataFrame(option_tree)
print(option_tree)

           0          1          2          3
0  11.043871  17.713888  27.631233  41.398246
1   0.000000   3.500654   6.545863  12.240090
2   0.000000   0.000000   0.000000   0.000000
3   0.000000   0.000000   0.000000   0.000000


**EUROPEAN CALL:**

In [8]:
def binomial_option_pricing_call_european(S, K, T, r, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    # Generación de precios finales del subyacente
    ST = np.array([S * (u ** j) * (d ** (N - j)) for j in range(N + 1)])
    
    # Payoff al vencimiento para CALL
    option_values = np.maximum(ST - K, 0)
    
    # Backward induction
    for i in range(N-1, -1, -1):
        option_values = np.exp(-r * dt) * (p * option_values[1:] + (1 - p) * option_values[:-1])
    
    return option_values[0]




99.99996504239652


**EUROPEAN PUT:**

In [9]:
def binomial_option_pricing_put_european(S, K, T, r, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    # Generación de precios finales del subyacente
    ST = np.array([S * (u**j) * (d**(N - j)) for j in range(N + 1)])
    
    # Payoff al vencimiento para PUT
    option_values = np.maximum(K - ST, 0)
    
    # Backward induction
    for i in range(N-1, -1, -1):
        option_values = np.exp(-r * dt) * (p * option_values[1:] + (1 - p) * option_values[:-1])
    
    return option_values[0]


**AMERICAN CALL:**

In [10]:
def binomial_option_pricing_call_american(S, K, T, r, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    ST = np.array([S * (u**j) * (d**(N - j)) for j in range(N + 1)])
    option_values = np.maximum(ST - K, 0)
    
    for i in range(N-1, -1, -1):
        ST = ST[:i+1] * u**-1  # recalcular los precios del subyacente en el paso i
        continuation = np.exp(-r * dt) * (p * option_values[1:] + (1 - p) * option_values[:-1])
        exercise = np.maximum(ST - K, 0)
        option_values = np.maximum(continuation, exercise)
    
    return option_values[0]

**AMERICAN PUT:**

In [11]:
def binomial_option_pricing_put_american(S, K, T, r, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    ST = np.array([S * (u**j) * (d**(N - j)) for j in range(N + 1)])
    option_values = np.maximum(K - ST, 0)
    
    for i in range(N-1, -1, -1):
        ST = ST[:i+1] * u**-1
        continuation = np.exp(-r * dt) * (p * option_values[1:] + (1 - p) * option_values[:-1])
        exercise = np.maximum(K - ST, 0)
        option_values = np.maximum(continuation, exercise)
    
    return option_values[0]